In [ ]:
import numpy as np
import scipy.optimize as opt
import scipy.stats as stats
from csv import DictReader
import matplotlib.pyplot as plt



# Данные
x_data = np.array([])
y_data = np.array([])

with open('diabetes.csv', 'r') as f:
    reader = DictReader(f)
    for row in reader:
        if row['Insulin'] == '0':
            x_data = np.append(x_data, int(row['Age']))
        else:
            y_data = np.append(y_data, int(row['Age']))


min_len = min(len(x_data), len(y_data))
x_data = x_data[:min_len]
y_data = y_data[:min_len]



print(x_data)
print(y_data)

split_idx = int(len(x_data) * 0.7)
x_calc, x_control = x_data[:split_idx], x_data[split_idx:]
y_calc, y_control = y_data[:split_idx], y_data[split_idx:]

# Модифицированный метод моментов — подбираем только b
def moments_estimation(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)

    def to_minimize(b):
        a = y_mean / (x_mean ** b)
        y_model = a * x ** b
        return np.sum((y - y_model) ** 2)

    res = opt.minimize_scalar(to_minimize, bounds=(0, 5), method='bounded')
    b_est = res.x
    a_est = y_mean / (x_mean ** b_est)
    return a_est, b_est

a_est, b_est = moments_estimation(x_calc, y_calc)
print(f"Оценённые параметры: a = {a_est:.4f}, b = {b_est:.4f}")

# Построение графика
x_plot = np.linspace(min(x_data), max(x_data), 100)
y_plot = a_est * x_plot ** b_est

plt.scatter(x_data, y_data, label='Эксперимент')
plt.plot(x_plot, y_plot, 'r-', label=f'Модель y = {a_est:.2f} * x^{b_est:.2f}')
plt.xlabel('Температура (x)')
plt.ylabel('Продажи мороженого (y)')
plt.legend()
plt.grid(True)
plt.show()

# Проверка по Колмогорову-Смирнову
y_pred_control = a_est * x_control ** b_est
ks_stat, p_value = stats.ks_2samp(y_control, y_pred_control)
print(f"KS Статистика: {ks_stat:.4f}, p-value: {p_value:.4f}")

if p_value > 0.05:
    print("Гипотеза НЕ отвергается. Модель адекватна.")
else:
    print("Гипотеза отвергается. Модель неадекватна.")

: 